In [ ]:
# Colab 로컬 연결
jupyter-lab --allow-root --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

In [ ]:
# 기본 설정
import numpy as np
import cv2
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tkinter as tk
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU

from sklearn.metrics import accuracy_score # LOOCY 모델 사용을 위해 추가
from sklearn.model_selection import train_test_split
from tkinter import messagebox
from PIL import Image, ImageTk

In [ ]:
# 데이터세트 경로 설정
dataset_path = 'C:/Users/prett/Desktop/IMS/인공지능/Dataset_0114(제공)/Dataset'

# 이미지 파일 목록과 라벨 생성
image_files = []
labels = []
for filename in os.listdir(dataset_path):
  if filename.endswith(('.png','.jpg')):
    image_files.append(os.path.join(dataset_path, filename))
    if 'o' in filename:
      labels.append(0)
    elif 'x' in filename:
      labels.append(1)
    else:
      labels.append(2)

# 디버깅
#print(len(image_files))

# 이미지 데이터세트 생성 (tf.data.Dataset 사용)
image_size = (300, 300)
BATCH_SIZE = 8
BUFFER_SIZE = tf.data.AUTOTUNE

# 이미지 전처리
def preprocess_image(image_path, target_size = (300, 300)):
  img = tf.keras.preprocessing.image.load_img(image_path, target_size = target_size)
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = img / 255.0
  return img

# 이미지 데이터와 라벨을 Numpy 배열로 변환
images = np.array([preprocess_image(image_file) for image_file in image_files])
labels = np.array(labels)

# 디버깅
'''
print(images.shape)
print(labels.shape)
print(images[0])
print(labels[0])
'''

# 훈련 데이터세트, 테스트 데이터세트로 나누기
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size = 0.2, random_state = 42)

# 디버깅
'''
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
'''

# 모델 구성 : 컨볼루션 추가
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(3, activation='softmax')
])

# 모델 컴파일
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

# 정확도 확인
_, accuracy = model.evaluate(x_test, y_test)
print(f'accuracy: {accuracy}')
print(f'최적 모델 : {accuracy*100}%')


C:\Users\prett\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.4751 - loss: 1.3988 - val_accuracy: 0.5139 - val_loss: 0.7006
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.5125 - loss: 0.7029 - val_accuracy: 0.6944 - val_loss: 0.6214
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6175 - loss: 0.6142 - val_accuracy: 0.8056 - val_loss: 0.4669
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.7988 - loss: 0.4280 - val_accuracy: 0.7917 - val_loss: 0.4329
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8605 - loss: 0.3215 - val_accuracy: 0.8194 - val_loss: 0.3786
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8885 - loss: 0.2655 - val_accuracy: 0.8611 - val_loss: 0.4277
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9402 - loss: 0.1955 - val_accuracy: 0.8056 - val_loss: 0.3706
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.9235 - loss: 0.2109 - val_accuracy: 0.8472 - val_loss: 0.3884
Epoch 9/

In [ ]:
# k-fold 적용 후 정확도 산출_기본 설정
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

# 5겹 교차 검증
kfold = KFold(n_splits=5, shuffle=True, random_state=22)

# 정확도 저장 리스트 초기화
accuracy_scores = []
# k-fold 수행
for train_index, test_index in kfold.split(images, labels):
  x_train, x_test = images[train_index], images[test_index]
  y_train, y_test = labels[train_index], labels[test_index]

# 모델 구성2 : 컨볼루션 추가(정확도가 높았음)
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), strides=(1, 1), activation='relu', input_shape=(300,300, 3)),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(128, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(256, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Conv2D(512, (3, 3), strides=(1, 1), activation='relu'),
    layers.MaxPooling2D((2, 2), padding='same'),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(3, activation='softmax')
])
# strides=(2, 2)로 설정시 padding=same으로 설정해서 컨볼루션 연산 불가능 -> (1, 1)로 설정

# 모델 컴파일
model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test))

# 예측
y_pred = np.argmax(model.predict(x_test), axis=1)

# 정확도 계산 후 저장
accuracy = accuracy_score(y_test, y_pred)
accuracy_scores.append(accuracy)

# 평균 정확도 출력
print(f'accuracy : {np.mean(accuracy_scores)}')
print(f'k-fold / stride  적용 정확도 : {np.mean(accuracy_scores)*100}%')

Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - accuracy: 0.4877 - loss: 1.3188 - val_accuracy: 0.5972 - val_loss: 0.7306
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5204 - loss: 0.7527 - val_accuracy: 0.8056 - val_loss: 0.6128
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.6866 - loss: 0.5702 - val_accuracy: 0.7778 - val_loss: 0.4189
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8323 - loss: 0.3388 - val_accuracy: 0.8611 - val_loss: 0.3253
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8620 - loss: 0.2956 - val_accuracy: 0.9028 - val_loss: 0.2326
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9223 - loss: 0.2001 - val_accuracy: 0.8750 - val_loss: 0.3320
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9490 - loss: 0.1087 - val_accuracy: 0.9028 - val_loss: 0.2465
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.9726 - loss: 0.0603 - val_accuracy: 0.8194 - val_loss: 0.6077
Epoch 9/

In [ ]:
# 코드 저장
model.save('model.h5')

# 미리 학습된 모델 불러오기
model.load_weights("model.h5")

In [ ]:
# 그림 그리기 위한 설정
drawing = False  # drawing : 마우스 왼쪽 버튼을 눌렀는지 나타내는 bool 변수 -> False로 초기화
ix, iy = -1, -1  # 마우스 왼쪽 버튼 눌렀을 때 좌표 저장(초기값 : -1)
img = np.full((350, 300, 3),255, np.uint8)  # 크기가 (350, 300)인 캔버스 생성 -> 버튼을 위해 x를 300 대신 350으로 설정
# img = np.zeros((350, 300, 3, np.uint8)) -> 검은색 캔버스

# 마우스 이벤트 처리 함수
def draw_circle(event, x, y, flags, param):
    global ix, iy, drawing, img
    # draw_circle 함수 내부에서 변수값들을 변경하고, 이 변경 사항들이 함수 외부에 유지되게 하기 위해 global 선언

    if event == cv2.EVENT_LBUTTONDOWN:  # 마우스 왼쪽 버튼이 눌렸을 때 실행
        drawing = True   # 마우스 왼쪽 버튼이 눌리면
        ix, iy = x, y    # 마우스 커서의 좌표를 ix, iy 변수에 저장

        # 버튼 클릭 로직 (버튼 영역 클릭했는지 확인)
        if 10 <= x <= 110 and 300 <= y <= 350:  # 확인 버튼 영역
            predict_image()
        elif 190 <= x <= 290 and 300 <= y <= 350:  # 지우기 버튼 영역
            clear_image()

    elif event == cv2.EVENT_MOUSEMOVE:  # 마우스 커서가 움직일 때 실행
        if drawing:
            cv2.line(img, (ix, iy), (x, y), (0, 0, 0), 10)   #(255,255,255) : 흰색
            ix, iy = x, y
    elif event == cv2.EVENT_LBUTTONUP:  # 마우스 왼쪽 버튼을 떼면
        drawing = False                 # drawing을 False로 설정 -> 그림그리기 끝!


# 예측 실행 함수
def predict_image():
    # 이미지 전처리
    input_img = cv2.resize(img, (300, 300))        # 그린 그림 크기를 (300, 300)으로 조정
    input_img = input_img / 255.0                  # 이미지의 픽셀 값을 0~1 사이로 정규화 -> 모델 학습 안정성, 성능 향상에 도움
    input_img = input_img.reshape(1, 300, 300, 3)  # 이미지 한개, 크기 300 x 300, RGB 세개의 색상 채널

    # 예측
    prediction = model.predict(input_img)   # 전처리된 이미지를 학습 모델에 입력
    predicted_class = np.argmax(prediction) # 예측 결과 중 가장 높은 확률 클래스 선택

    # 결과 출력
    if predicted_class == 0:
        result_text = "You Draw O"
    elif predicted_class == 1:
        result_text = "You Draw X"
    else:
        result_text = "Draw O or X"


    cv2.putText(img, result_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
    # 결과 텍스트 이미지를 초록색으로 표시 (0, 2550, 0) -> 크기 1, 녹색, 글씨 두께
    cv2.imshow('Draw O or X', img) # 결과가 추가된 이미지를 캔버스에 표시

# 화면 지우기 함수
def clear_image():   # 함수 선언 -> 그린 그림 지우기
    global img
    img[:300, :] = np.full((300, 300, 3),255, np.uint8)  # 그림 영역만 초기화 -> 캔버스를 초기화 시 버튼도 사라짐
    # 그림 영역만 검은색으로 초기화
    cv2.imshow("Draw O or X", img)

# OpenCV 창 생성 및 이벤트 설정
cv2.namedWindow('Draw O or X')                    # OpenCV 창 생성 -> 그림 그리고 결과 확인
cv2.setMouseCallback('Draw O or X', draw_circle)  # 마우스 이벤트 발생 시 draw_circle 호출

# 버튼 생성
def button_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        if 10 <= x <= 110 and 300 <= y <= 350:     # 확인 버튼 영역
            predict_image()                        # 함수 실행 -> 그림 예측
        elif 190 <= x <= 290 and 300 <= y <= 350:  # 지우기 버튼 영역
            clear_image()                          # 함수 실행 -> 캔버스 화면 초기화

cv2.namedWindow('Draw O or X')                   # 창 생성
cv2.setMouseCallback('Draw O or X', draw_circle) # draw_circle 함수를 호출

# 버튼 그리기 (while 루프 안에서 반복)
while True:
    # 버튼 영역 표시
    #cv2.rectangle(img, (8, 298), (108, 348), (128,128, 128), 2) # 그림자 색상(128, 128, 128)
    cv2.rectangle(img, (10, 300), (110, 350), (255, 0, 0), -1)  # 확인 버튼
    cv2.rectangle(img, (190, 300), (290, 350), (0, 0, 255), -1)  # 지우기 버튼
    cv2.putText(img, "Save", (45, 330), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    cv2.putText(img, "Delete", (220, 330), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    cv2.imshow('Draw O or X', img)   # 버튼이 그려진 이미지를 창에 표시
    k = cv2.waitKey(1) & 0xFF        # 키 입력 기다리기

    if k == 27:  # ESC 키를 누르면 종료 -> ASCII 코드 27
        break

cv2.destroyAllWindows()   # 창 닫기

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
